```python
def sha1(data: bytes) -> bytes
def sha256(data: bytes) -> bytes
def sha512(data: bytes) -> bytes
def ripemd160(data: bytes) -> bytes
def recover_key(data: bytes) -> bytes

def assert_sha1(data: bytes, hash: bytes)
def assert_sha256(data: bytes, hash: bytes)
def assert_sha512(data: bytes, hash: bytes)
def assert_ripemd160(data: bytes, hash: bytes)
def assert_recover_key(data: bytes, hash: bytes)
```

In [1]:
from init import *

In [2]:
code = '''
import chain
def apply(receiver, code, action):
    print(chain.sha1(b'helloworld'))
    chain.assert_sha1(b'helloworld', chain.sha1(b'helloworld'))
    chain.assert_sha256(b'helloworld', chain.sha256(b'helloworld'))
    chain.assert_sha512(b'helloworld', chain.sha512(b'helloworld'))
    chain.assert_ripemd160(b'helloworld', chain.ripemd160(b'helloworld'))
'''

code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)
r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


b'j\xdf\xb1\x83\xa4\xa2\xc9J/\x92\xda\xb5\xad\xe7b\xa4x\x89\xa5\xa1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [3]:
uuosapi.get_account(python_contract)

{'account_name': 'hello',
 'head_block_num': 3296,
 'head_block_time': '2021-02-07T07:28:12.000',
 'privileged': False,
 'last_code_update': '2021-02-07T05:46:20.000',
 'created': '2021-02-07T05:31:46.000',
 'core_liquid_balance': '10000000.1000 EOS',
 'ram_quota': -1,
 'net_weight': -1,
 'cpu_weight': -1,
 'net_limit': {'used': -1,
  'available': -1,
  'max': -1,
  'last_usage_update_time': {'slot': 332999046},
  'current_used': -1},
 'cpu_limit': {'used': -1,
  'available': -1,
  'max': -1,
  'last_usage_update_time': {'slot': 332999046},
  'current_used': -1},
 'ram_usage': 2715211,
 'permissions': [{'perm_name': 'active',
   'parent': 'owner',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV',
      'weight': 1}],
    'accounts': [{'permission': {'actor': 'hello', 'permission': 'eosio.code'},
      'weight': 1}],
    'waits': []}},
  {'perm_name': 'owner',
   'parent': '',
   'required_auth': {'threshold': 1,
    'keys

In [5]:
import base58
public_key = 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV'

print(public_key)
b'\x00'+base58.b58decode(public_key[3:])

EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV


b'\x00\x02\xa8\x91\xe0\xddW\x13.\xd6\x83\xbc\x87]\xac\xc9a\xc6\xfd]\xfa\xe6\x80\x0b\xc6\x18\x1a\xb6\x8b\xb8H%\x1eR\xaa<\x99\xa6'

```python
def assert_recover_key(digest, sign, public_key) -> None
def recover_key(digest, sign) -> bytes
```

In [7]:

code = r'''
import struct
import chain
public_key = b'\x00\x02\xa8\x91\xe0\xddW\x13.\xd6\x83\xbc\x87]\xac\xc9a\xc6\xfd]\xfa\xe6\x80\x0b\xc6\x18\x1a\xb6\x8b\xb8H%\x1eR\xaa<\x99\xa6'

def apply(receiver, code, action):
    data = chain.read_action_data()
    value, size = struct.unpack_uint32(data)
    text = data[size:value+size]
    sign = data[size+value:]
    print(text)
    print(data)
    print(len(public_key))

    digest = chain.sha256(text)
    chain.assert_recover_key(digest, sign, public_key)
    
    key = chain.recover_key(digest, sign)
    print(key)
    print(len(key))
    
    checksum = chain.ripemd160(key[1:])
    print(checksum)
'''
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"text", "type":"bytes"},
        {"name":"sign", "type":"signature"}
        ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''


uuosapi.set_abi(python_contract, abi)

code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)


text = b'hello,world'
h = hashlib.sha256()
h.update(text)
signature = wallet.sign_digest(h.digest(), 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV')

args = {'text': text.hex(), 'sign': signature}

r = uuosapi.push_action(python_contract, 'sayhello', args, {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])

b'hello,world'
b'\x0bhello,world\x00  \x87\xd1\x9b\xca|L\xf3\x1e\x8d\x88.\xf7\x1e\xa2\xcc\xab\x7f%\xd5\xc0\xd4x\x01\xdeK\xeb:\x0e\x1d\xd9\x87/\xf7\xd0\x06b<\x95\xaf\xba\xcf;,n7u\xff\xc5>&Y\xad\xeb<\xd7c&\x03\x00\x81\xb2\xb9\xc5'
38
b'\x00\x02\xa8\x91\xe0\xddW\x13.\xd6\x83\xbc\x87]\xac\xc9a\xc6\xfd]\xfa\xe6\x80\x0b\xc6\x18\x1a\xb6\x8b\xb8H%\x1eR'
34
b'\xaa<\x99\xa6#\rn$\xaa\xa0\x95LV\xcfl\x8d\xf3e\xf8\xb3\xfa\xe6\x80\x0b\x01\x00\x00\x00\x01\x00\x00\x00'



In [8]:
import base58
public_key = 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV'
print(public_key)
raw_pub_key = base58.b58decode(public_key[3:])[:-4]
print(raw_pub_key)

EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV
b'\x02\xa8\x91\xe0\xddW\x13.\xd6\x83\xbc\x87]\xac\xc9a\xc6\xfd]\xfa\xe6\x80\x0b\xc6\x18\x1a\xb6\x8b\xb8H%\x1eR'


In [10]:
import base58
h = hashlib.new('ripemd160')
h.update(raw_pub_key)

digest = h.digest()

public_key = raw_pub_key + digest[:4]
b'EOS'+base58.b58encode(public_key)


b'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV'